- 잠재공간을 이용하는 두가지  생성 모델
  - 변이형 오토인코더(VAE)
  - 생성적 적대 신경망(GAN)

- 자기회귀모델(autoregressive model)
  - 잠재 확률변수가 아니라 시퀀스에 있는 이전값을 바탕으로 예측을 만듦
  

In [1]:
# 자기회귀모델 LSTM(텍스트 데이터),  PixelCNN(이미지 데이터)

In [2]:
# https://drive.google.com/file/d/1CPCu895GkGPrvChspl7iPBkoqpBzKCOO/view?usp=sharing

In [3]:
!unzip '/content/drive/MyDrive/레시피데이터셋/Epicurious - Recipes with Rating and Nutrition.zip'

Archive:  /content/drive/MyDrive/레시피데이터셋/Epicurious - Recipes with Rating and Nutrition.zip
  inflating: epi_r.csv               
  inflating: full_format_recipes.json  
  inflating: recipe.py               
  inflating: utils.py                


In [5]:
import pandas as pd
import numpy as np
import json

In [9]:
with open('/content/full_format_recipes.json')as json_data:
  recipe_data =  json.load(json_data)
recipe_data[0].keys()


dict_keys(['directions', 'fat', 'date', 'categories', 'calories', 'desc', 'protein', 'rating', 'title', 'ingredients', 'sodium'])

In [14]:
recipe_data[0]['title'],recipe_data[0]['directions']

('Lentil, Apple, and Turkey Wrap ',
 ['1. Place the stock, lentils, celery, carrot, thyme, and salt in a medium saucepan and bring to a boil. Reduce heat to low and simmer until the lentils are tender, about 30 minutes, depending on the lentils. (If they begin to dry out, add water as needed.) Remove and discard the thyme. Drain and transfer the mixture to a bowl; let cool.',
  '2. Fold in the tomato, apple, lemon juice, and olive oil. Season with the pepper.',
  '3. To assemble a wrap, place 1 lavash sheet on a clean work surface. Spread some of the lentil mixture on the end nearest you, leaving a 1-inch border. Top with several slices of turkey, then some of the lettuce. Roll up the lavash, slice crosswise, and serve. If using tortillas, spread the lentils in the center, top with the turkey and lettuce, and fold up the bottom, left side, and right side before rolling away from you.'])

In [15]:
temp = recipe_data[0]
temp['title']+" : " + ' '.join(temp['directions'])

'Lentil, Apple, and Turkey Wrap  : 1. Place the stock, lentils, celery, carrot, thyme, and salt in a medium saucepan and bring to a boil. Reduce heat to low and simmer until the lentils are tender, about 30 minutes, depending on the lentils. (If they begin to dry out, add water as needed.) Remove and discard the thyme. Drain and transfer the mixture to a bowl; let cool. 2. Fold in the tomato, apple, lemon juice, and olive oil. Season with the pepper. 3. To assemble a wrap, place 1 lavash sheet on a clean work surface. Spread some of the lentil mixture on the end nearest you, leaving a 1-inch border. Top with several slices of turkey, then some of the lettuce. Roll up the lavash, slice crosswise, and serve. If using tortillas, spread the lentils in the center, top with the turkey and lettuce, and fold up the bottom, left side, and right side before rolling away from you.'

In [17]:
filtered_data = []
for rd in recipe_data:
  if ('title' in rd) and (rd['title'] is not None) and (rd['directions'] is not None):
    filtered_data.append( rd['title']+" : " + ' '.join(rd['directions']) )

In [19]:
filtered_data[:3]

['Lentil, Apple, and Turkey Wrap  : 1. Place the stock, lentils, celery, carrot, thyme, and salt in a medium saucepan and bring to a boil. Reduce heat to low and simmer until the lentils are tender, about 30 minutes, depending on the lentils. (If they begin to dry out, add water as needed.) Remove and discard the thyme. Drain and transfer the mixture to a bowl; let cool. 2. Fold in the tomato, apple, lemon juice, and olive oil. Season with the pepper. 3. To assemble a wrap, place 1 lavash sheet on a clean work surface. Spread some of the lentil mixture on the end nearest you, leaving a 1-inch border. Top with several slices of turkey, then some of the lettuce. Roll up the lavash, slice crosswise, and serve. If using tortillas, spread the lentils in the center, top with the turkey and lettuce, and fold up the bottom, left side, and right side before rolling away from you.',
 'Boudin Blanc Terrine with Red Onion Confit  : Combine first 9 ingredients in heavy medium saucepan. Add 3 shallo

- 텍스트를 토큰화
  - 단어나 문자로 나누는 작업
- 단어 토큰
  - 모든 텍스트를 소문자로 변환
  - 어휘사전(vocabulary)이 매우 클 수도 있음
  - 어간(stem) : 현재 과거, 진행
  - 마침표와 쉼표를 제거
  - 훈련어휘에 없는 단어는 예측 불가
- 문자 토큰
  - 시퀀스를 생성해서 훈련 어휘에 없는 새로운 단어를 생성가능
  - 어휘사전이 매우 작아진다 그래서 마지막 출력층에 학습할 가중치가 적기때문에 훈련속도에 유리

In [24]:
# 텍스트 전처리
import string
import re
def pad_punctuation(s):
  s = re.sub(f"([{string.punctuation}])", r' \1', s)
  s = re.sub(' +',' ',s)
  return s
pad_punctuation("!!!abc#$")

' ! ! !abc # $'

In [21]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
text_data = [pad_punctuation(x) for x in filtered_data]
text_data[:5]

In [26]:
# 텐서플로의 데이터셋으로 변환
import tensorflow as  tf

In [28]:
text_ds = tf.data.Dataset.from_tensor_slices(text_data).batch(32).shuffle(1000)

In [29]:
import tensorflow.keras.layers as layers

In [30]:
vectorize_layer =  layers.TextVectorization(
    standardize= 'lower',  # 소문자로 변경
    max_tokens = 10000, # 자주 등장하는 1000개의 단어에 정수를 부여
    output_mode = 'int',
    output_sequence_length = 200+1  # 시퀀스의 길이가 201개의 토큰으로 자르거나 패팅
)